In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Machine Learning/transformer')
os.getcwd()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/drive/My Drive/Machine Learning/transformer'

# EDA (Exploratory Data Analysis) & Preprocessing

In [2]:
df = pd.read_csv("./Data/smile-annotations-final.csv", names = ["id", "text", "category"])
df.set_index("id", inplace=True)

In [3]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [4]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [5]:
df = df[~df.category.str.contains("\|")]
df = df[df.category!="nocode"]

In [6]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [7]:
possible_labels = df.category.unique()

In [8]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label] = index

In [9]:
df["label"] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


# Training / Validation split

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15, 
    random_state = 17,
    stratify = df.label.values
)

In [11]:
df["data_type"] = ["not set"]*df.shape[0]

In [12]:
df.loc[X_train, "data_type"] = "train"
df.loc[X_val, "data_type"] = "val"

In [13]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train


In [14]:
df.groupby(["category", "label", "data_type"]).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# Loading tokenizer and Encoding data

In [15]:
! pip install transformers

In [16]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [17]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True,
)

In [18]:
from torch import Tensor
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=="train"].text.values, 
    add_special_tokens = True,
    return_attention_mask = True,
    padding = True,
    truncation=True,
    max_length = 256,
    return_tensors = "pt"
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=="val"].text.values, 
    add_special_tokens = True,
    return_attention_mask = True,
    truncation=True,
    padding = True,
    max_length = 256,
    return_tensors = "pt"
)

input_ids_train = encoded_data_train["input_ids"]
attention_mask_train = encoded_data_train["attention_mask"]
labels_train = Tensor(df[df.data_type=="train"].label.values)

input_ids_val = encoded_data_val["input_ids"]
attention_mask_val = encoded_data_val["attention_mask"]
labels_val = Tensor(df[df.data_type=="val"].label.values)

In [19]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_mask_train, labels_train)
dataset_val = TensorDataset(input_ids_val, 
                            attention_mask_val, labels_val)

# Setting up BERT pre-trained Model

In [20]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Creating Data Loaders

In [21]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16 #32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)
dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

# Setting up Optimizer and Scheduler



In [22]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [23]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, # 2e-5 > 5e-5
    eps = 1e-8
)

epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataset_train) * epochs
)

# Defining our performance Metrics

In [24]:
import numpy as np
from sklearn.metrics import f1_score

In [48]:
def f1_score_func(preds, labels):
    preds_flatten = np.argmax(preds, axis = 1).flatten()
    labels_flatten = labels.flatten()
    return f1_score(preds_flatten, labels_flatten, average="weighted")

def accuraccy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flatten = np.argmax(preds, axis = 1).flatten()
    labels_flatten = labels.flatten()

    for label in np.unique(labels_flatten):
        y_preds = preds_flatten[labels_flatten == label] 
        y_true = labels_flatten[labels_flatten == label] 
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Creating our training loop

In [26]:
from random import seed
import torch

seed_value = 17
seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

In [27]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")

model.to(device)
print(device)

cuda


In [28]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:
        batch = tuple(b.to(device).long() for b in batch)

        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2]}

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total / len(dataloader_val)
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [30]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, 
                        desc="Epoch{:1d}".format(epoch),
                        leave=False,
                        disable=False)
    
    for batch in progress_bar:
      
        model.zero_grad()
        
        batch = tuple(b.to(device).long() for b in batch)

        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            'labels':         batch[2]}

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item() 
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
        logits = outputs[1]
    torch.save(model.state_dict(), f'Model_BERT_ft_epoch{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training_loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'f1 score (weighted): {val_f1}')



Epoch {epoch}
Training_loss: 0.6302107353753681
Validation loss: 0.5524276665278843
f1 score (weighted): 0.8608122583104881



Epoch {epoch}
Training_loss: 0.45379414989412586
Validation loss: 0.530462133032935
f1 score (weighted): 0.8697297164841088



Epoch {epoch}
Training_loss: 0.3303221674470962
Validation loss: 0.5266962732587542
f1 score (weighted): 0.8943226781132348



Epoch {epoch}
Training_loss: 0.23420918897926052
Validation loss: 0.5278567331177848
f1 score (weighted): 0.890198965973303



Epoch {epoch}
Training_loss: 0.16948841380167612
Validation loss: 0.5410209814352649
f1 score (weighted): 0.8958081652536855



Epoch {epoch}
Training_loss: 0.10798090513017547
Validation loss: 0.5606195458344051
f1 score (weighted): 0.8700988798827503



Epoch {epoch}
Training_loss: 0.07436703218595136
Validation loss: 0.5622778620038714
f1 score (weighted): 0.8771420372535668



Epoch {epoch}
Training_loss: 0.05181750158567123
Validation loss: 0.6517665556498936
f1 score (weighted): 0.8759311001891745



Epoch {epoch}
Training_loss: 0.02782883321206215
Validation loss: 0.7503090884004321
f1 score (weighted): 0.8730453915497439



Epoch {epoch}
Training_loss: 0.02217345724259562
Validation loss: 0.7237548870699746
f1 score (weighted): 0.8656688703286819



# Loading & Evaluating our Model

In [33]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                        num_labels = len(label_dict),
                                                        output_attentions = False,
                                                        output_hidden_states = False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [34]:
model.to(device)
model.load_state_dict(torch.load('Model_BERT_ft_epoch10.model', map_location=torch.device('cuda')))

In [50]:
_, predictions, true_vals = evaluate(dataloader_val)
accuraccy_per_class(predictions, true_vals)
f1_score_func(predictions, true_vals)

Class: happy
Accuracy: 162/171

Class: not-relevant
Accuracy: 19/32

Class: angry
Accuracy: 7/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 2/5

Class: surprise
Accuracy: 2/5



0.8656688703286819